In [3]:
!pip install pandas
import os
import json
import pandas as pd

# Data Extraction - Aggregated Transaction/Insurance/user
# General function to process JSON data
def extract_data(base_path, columns, data_key, name_key, count_key, amount_key=None):
    data_dict = {col: [] for col in columns}

    state_list = os.listdir(base_path)

    for state in state_list:
        p_state = os.path.join(base_path, state)
        if not os.path.isdir(p_state):  # Skip non-directory files
            continue

        year_list = os.listdir(p_state)

        for year in year_list:
            p_year = os.path.join(p_state, year)
            if not os.path.isdir(p_year):
                continue

            quarter_list = os.listdir(p_year)

            for quarter in quarter_list:
                p_quarter = os.path.join(p_year, quarter)
                if not quarter.endswith('.json'):  # Ensure it's a JSON file
                    continue

                # Read JSON file
                with open(p_quarter, 'r', encoding='utf-8') as file:
                    try:
                        D = json.load(file)
                        extracted_data = D.get('data', {}).get(data_key, [])

                        for entry in extracted_data:
                            data_dict['State'].append(state)
                            data_dict['Year'].append(year)
                            data_dict['Quarter'].append(int(quarter.replace('.json', '')))
                            data_dict[name_key].append(entry.get('name', None))
                            data_dict[count_key].append(entry.get('paymentInstruments', [{}])[0].get('count', 0))
                            if amount_key:
                                data_dict[amount_key].append(entry.get('paymentInstruments', [{}])[0].get('amount', 0))

                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON in {p_quarter}: {e}")

    return pd.DataFrame(data_dict)


# Extract Aggregated Transactions Data
Agg_Trans = extract_data(
    base_path=r"C:\Users\sindh\OneDrive\Desktop\02_Phonepe\data\data\aggregated\transaction\country\india\state",
    columns=['State', 'Year', 'Quarter', 'Transaction_type', 'Transaction_count', 'Transaction_amount'],
    data_key='transactionData',
    name_key='Transaction_type',
    count_key='Transaction_count',
    amount_key='Transaction_amount'
)

# Extract Aggregated Insurance Data
Agg_Ins = extract_data(
    base_path=r"C:\Users\sindh\OneDrive\Desktop\02_Phonepe\data\data\aggregated\insurance\country\india\state",
    columns=['State', 'Year', 'Quarter', 'Category', 'Insurance_count', 'Insurance_amount'],
    data_key='transactionData',
    name_key='Category',
    count_key='Insurance_count',
    amount_key='Insurance_amount'
)

# Extract Aggregated User Data (Different structure)
def extract_user_data(base_path):
    columns = ['State', 'Year', 'Quarter', 'Registered_Users', 'App_Opens']
    data_dict = {col: [] for col in columns}

    state_list = os.listdir(base_path)

    for state in state_list:
        p_state = os.path.join(base_path, state)
        if not os.path.isdir(p_state):
            continue

        year_list = os.listdir(p_state)

        for year in year_list:
            p_year = os.path.join(p_state, year)
            if not os.path.isdir(p_year):
                continue

            quarter_list = os.listdir(p_year)

            for quarter in quarter_list:
                p_quarter = os.path.join(p_year, quarter)
                if not quarter.endswith('.json'):
                    continue

                with open(p_quarter, 'r', encoding='utf-8') as file:
                    try:
                        D = json.load(file)
                        aggregated_data = D.get('data', {}).get('aggregated', {})

                        data_dict['State'].append(state)
                        data_dict['Year'].append(year)
                        data_dict['Quarter'].append(int(quarter.replace('.json', '')))
                        data_dict['Registered_Users'].append(aggregated_data.get('registeredUsers', 0))
                        data_dict['App_Opens'].append(aggregated_data.get('appOpens', 0))

                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON in {p_quarter}: {e}")

    return pd.DataFrame(data_dict)


Agg_User = extract_user_data(
    base_path=r"C:\Users\sindh\OneDrive\Desktop\02_Phonepe\data\data\aggregated\user\country\india\state"
)

# Set display format
pd.options.display.float_format = '{:.2f}'.format

# Print DataFrames (Optional)
Agg_Trans




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1845307.47
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,12138655.30
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,452507.17
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,10601.42
4,andaman-&-nicobar-islands,2018,1,Others,256,184689.87
...,...,...,...,...,...,...
5029,west-bengal,2024,4,Merchant payments,655100809,389286175909.00
5030,west-bengal,2024,4,Peer-to-peer payments,493217788,1361927136312.00
5031,west-bengal,2024,4,Recharge & bill payments,76043195,57534056540.00
5032,west-bengal,2024,4,Financial Services,2352084,847296537.00


In [4]:
Agg_Ins

,State,Year,Quarter,Category,Insurance_count,Insurance_amount
0,andaman-&-nicobar-islands,2020,2,Insurance,6,1360.00
1,andaman-&-nicobar-islands,2020,3,Insurance,41,15380.00
2,andaman-&-nicobar-islands,2020,4,Insurance,124,157975.00
3,andaman-&-nicobar-islands,2021,1,Insurance,225,244266.00
4,andaman-&-nicobar-islands,2021,2,Insurance,137,181504.00
...,...,...,...,...,...,...
677,west-bengal,2023,4,Insurance,72712,100365562.00
678,west-bengal,2024,1,Insurance,79576,104987909.00
679,west-bengal,2024,2,Insurance,67048,89476633.00
680,west-bengal,2024,3,Insurance,77158,107451766.00


In [5]:
Agg_User

,State,Year,Quarter,Registered_Users,App_Opens
0,andaman-&-nicobar-islands,2018,1,6740,0
1,andaman-&-nicobar-islands,2018,2,9405,0
2,andaman-&-nicobar-islands,2018,3,12149,0
3,andaman-&-nicobar-islands,2018,4,15222,0
4,andaman-&-nicobar-islands,2019,1,18596,0
...,...,...,...,...,...
1003,west-bengal,2023,4,30064546,1467442959
1004,west-bengal,2024,1,31306843,633526507
1005,west-bengal,2024,2,32540397,704276274
1006,west-bengal,2024,3,33612828,709864323


In [6]:
#Data Extraction - Map Transaction/Insurance/user
# General function to extract data from JSON files
def extract_map_data(base_path, columns, data_key, name_key, count_key, amount_key=None, is_user_data=False):
    data_dict = {col: [] for col in columns}
    
    # Get list of states
    state_list = os.listdir(base_path)

    for state in state_list:
        p_state = os.path.join(base_path, state)
        if not os.path.isdir(p_state):  # Ensure it's a directory
            continue

        year_list = os.listdir(p_state)

        for year in year_list:
            p_year = os.path.join(p_state, year)
            if not os.path.isdir(p_year):
                continue

            quarter_list = os.listdir(p_year)

            for quarter in quarter_list:
                p_quarter = os.path.join(p_year, quarter)
                if not quarter.endswith('.json'):
                    continue

                # Open and load JSON file
                with open(p_quarter, 'r', encoding='utf-8') as file:
                    try:
                        D = json.load(file)
                        
                        if is_user_data:
                            hover_data = D.get('data', {}).get(data_key, {})
                            for district, values in hover_data.items():
                                data_dict['District'].append(district)
                                data_dict['RegisteredUsers'].append(values.get('registeredUsers', 0))
                                data_dict['AppOpens'].append(values.get('appOpens', 0))
                                data_dict['State'].append(state)
                                data_dict['Year'].append(year)
                                data_dict['Quarter'].append(int(quarter.replace('.json', '')))
                        else:
                            hover_data_list = D.get('data', {}).get(data_key, [])
                            for entry in hover_data_list:
                                data_dict[name_key].append(entry.get('name', None))
                                data_dict[count_key].append(entry.get('metric', [{}])[0].get('count', 0))
                                if amount_key:
                                    data_dict[amount_key].append(entry.get('metric', [{}])[0].get('amount', 0))
                                data_dict['State'].append(state)
                                data_dict['Year'].append(year)
                                data_dict['Quarter'].append(int(quarter.replace('.json', '')))

                    except json.JSONDecodeError as e:
                        print(f"Error processing {p_quarter}: {e}")

    return pd.DataFrame(data_dict)

# Extract Map Transactions Data
Map_Trans = extract_map_data(
    base_path=r"C:\Users\sindh\OneDrive\Desktop\02_Phonepe\data\data\map\transaction\hover\country\india\state",
    columns=['State', 'Year', 'Quarter', 'Transaction_type', 'Transaction_count', 'Transaction_amount'],
    data_key='hoverDataList',
    name_key='Transaction_type',
    count_key='Transaction_count',
    amount_key='Transaction_amount'
)

# Extract Map Insurance Data
Map_Insurance = extract_map_data(
    base_path=r"C:\Users\sindh\OneDrive\Desktop\02_Phonepe\data\data\map\insurance\hover\country\india\state",
    columns=['State', 'Year', 'Quarter', 'District', 'Insurance_count', 'Insurance_amount'],
    data_key='hoverDataList',
    name_key='District',
    count_key='Insurance_count',
    amount_key='Insurance_amount'
)

# Extract Map Users Data
Map_User = extract_map_data(
    base_path=r"C:\Users\sindh\OneDrive\Desktop\02_Phonepe\data\data\map\user\hover\country\india\state",
    columns=['State', 'Year', 'Quarter', 'District', 'RegisteredUsers', 'AppOpens'],
    data_key='hoverData',
    name_key=None,  # Not needed for user data
    count_key=None,  # Not needed for user data
    amount_key=None,  # Not needed for user data
    is_user_data=True
)

# Set display format
pd.options.display.float_format = '{:.2f}'.format

# Print DataFrames (Optional)
Map_Trans


,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,931663.08
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,12560249.34
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1139848.80
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1317863.08
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,23948235.52
...,...,...,...,...,...,...
20599,west-bengal,2024,4,alipurduar district,15875637,20992506730.00
20600,west-bengal,2024,4,paschim bardhaman district,56616799,69687350676.00
20601,west-bengal,2024,4,nadia district,65274337,107932013030.00
20602,west-bengal,2024,4,birbhum district,36905213,57787007226.00


In [7]:
Map_Insurance

,State,Year,Quarter,District,Insurance_count,Insurance_amount
0,andaman-&-nicobar-islands,2020,2,south andaman district,3,795.00
1,andaman-&-nicobar-islands,2020,2,nicobars district,3,565.00
2,andaman-&-nicobar-islands,2020,3,north and middle andaman district,1,281.00
3,andaman-&-nicobar-islands,2020,3,south andaman district,35,13651.00
4,andaman-&-nicobar-islands,2020,3,nicobars district,5,1448.00
...,...,...,...,...,...,...
13871,west-bengal,2024,4,alipurduar district,1023,1613143.00
13872,west-bengal,2024,4,paschim bardhaman district,4945,7005851.00
13873,west-bengal,2024,4,nadia district,3807,5031294.00
13874,west-bengal,2024,4,birbhum district,1818,2423290.00


In [8]:
Map_User


,State,Year,Quarter,District,RegisteredUsers,AppOpens
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
20603,west-bengal,2024,4,alipurduar district,475688,31842355
20604,west-bengal,2024,4,paschim bardhaman district,1468252,80543469
20605,west-bengal,2024,4,nadia district,1861738,98740305
20606,west-bengal,2024,4,birbhum district,1114220,73465525


In [9]:
#Data Extraction - Top Transaction/Insurance/user
# General function to extract data
def extract_top_data(base_path, columns, data_key, name_key, count_key, amount_key=None, is_user_data=False):
    data_dict = {col: [] for col in columns}

    state_list = os.listdir(base_path)
    
    for state in state_list:
        p_state = os.path.join(base_path, state)
        if not os.path.isdir(p_state):  # Ensure it's a directory
            continue
        
        year_list = os.listdir(p_state)
        
        for year in year_list:
            p_year = os.path.join(p_state, year)
            if not os.path.isdir(p_year):
                continue
            
            quarter_list = os.listdir(p_year)
            
            for quarter in quarter_list:
                p_quarter = os.path.join(p_year, quarter)
                if not quarter.endswith('.json'):
                    continue
                
                with open(p_quarter, 'r', encoding='utf-8') as file:
                    try:
                        D = json.load(file)

                        if is_user_data:
                            # Extract districts data
                            districts_data = D.get('data', {}).get('districts', [])
                            for district in districts_data:
                                data_dict['State'].append(state)
                                data_dict['Year'].append(year)
                                data_dict['Quarter'].append(int(quarter.replace('.json', '')))
                                data_dict['District'].append(district.get('name'))
                                data_dict['RegisteredUsers_District'].append(district.get('registeredUsers', 0))
                                data_dict['Pincode'].append(None)
                                data_dict['RegisteredUsers_Pincode'].append(None)

                            # Extract pincodes data
                            pincodes_data = D.get('data', {}).get('pincodes', [])
                            for pincode in pincodes_data:
                                data_dict['State'].append(state)
                                data_dict['Year'].append(year)
                                data_dict['Quarter'].append(int(quarter.replace('.json', '')))
                                data_dict['District'].append(None)
                                data_dict['RegisteredUsers_District'].append(None)
                                data_dict['Pincode'].append(pincode.get('name'))
                                data_dict['RegisteredUsers_Pincode'].append(pincode.get('registeredUsers', 0))

                        else:
                            districts_data = D.get('data', {}).get(data_key, [])
                            for district in districts_data:
                                data_dict[name_key].append(district.get('entityName', 'Unknown'))
                                metrics = district.get('metric', {})
                                data_dict[count_key].append(metrics.get('count', 0))
                                if amount_key:
                                    data_dict[amount_key].append(metrics.get('amount', 0.0))
                                data_dict['State'].append(state)
                                data_dict['Year'].append(year)
                                data_dict['Quarter'].append(int(quarter.replace('.json', '')))

                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON in {p_quarter}: {e}")

    return pd.DataFrame(data_dict)

# Extract Top Transactions Data
Top_Trans = extract_top_data(
    base_path=r"C:\Users\sindh\OneDrive\Desktop\02_Phonepe\data\data\top\transaction\country\india\state",
    columns=['State', 'Year', 'Quarter', 'District', 'Transaction_count', 'Transaction_amount'],
    data_key='districts',
    name_key='District',
    count_key='Transaction_count',
    amount_key='Transaction_amount'
)

# Extract Top Insurance Data
Top_Ins = extract_top_data(
    base_path=r"C:\Users\sindh\OneDrive\Desktop\02_Phonepe\data\data\top\insurance\country\india\state",
    columns=['State', 'Year', 'Quarter', 'District', 'Insurance_count', 'Insurance_amount'],
    data_key='districts',
    name_key='District',
    count_key='Insurance_count',
    amount_key='Insurance_amount'
)

# Extract Top User Data
Top_User = extract_top_data(
    base_path=r"C:\Users\sindh\OneDrive\Desktop\02_Phonepe\data\data\top\user\country\india\state",
    columns=['State', 'Year', 'Quarter', 'District', 'RegisteredUsers_District', 'Pincode', 'RegisteredUsers_Pincode'],
    data_key=None,
    name_key=None,
    count_key=None,
    amount_key=None,
    is_user_data=True
)

# Consolidate user data
Top_User["RegisteredUsers"] = Top_User["RegisteredUsers_District"].fillna(Top_User["RegisteredUsers_Pincode"])
Top_User.drop(columns=["RegisteredUsers_District", "RegisteredUsers_Pincode"], inplace=True)

# Set display format
pd.options.display.float_format = '{:.2f}'.format

# Print DataFrames (Optional)
Top_Trans

,State,Year,Quarter,District,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,south andaman,5688,12560249.34
1,andaman-&-nicobar-islands,2018,1,nicobars,528,1139848.80
2,andaman-&-nicobar-islands,2018,1,north and middle andaman,442,931663.08
3,andaman-&-nicobar-islands,2018,2,south andaman,9395,23948235.52
4,andaman-&-nicobar-islands,2018,2,nicobars,1120,3072436.91
...,...,...,...,...,...,...
8291,west-bengal,2024,4,howrah,67496284,102576375745.00
8292,west-bengal,2024,4,nadia,65274337,107932013030.00
8293,west-bengal,2024,4,malda,63424613,102421242316.00
8294,west-bengal,2024,4,hooghly,59676576,90713828065.00


In [10]:
Top_Ins


,State,Year,Quarter,District,Insurance_count,Insurance_amount
0,andaman-&-nicobar-islands,2020,2,nicobars,3,565.00
1,andaman-&-nicobar-islands,2020,2,south andaman,3,795.00
2,andaman-&-nicobar-islands,2020,3,south andaman,35,13651.00
3,andaman-&-nicobar-islands,2020,3,nicobars,5,1448.00
4,andaman-&-nicobar-islands,2020,3,north and middle andaman,1,281.00
...,...,...,...,...,...,...
5603,west-bengal,2024,4,paschim bardhaman,4945,7005851.00
5604,west-bengal,2024,4,paschim medinipur,4155,5405054.00
5605,west-bengal,2024,4,malda,4042,4902098.00
5606,west-bengal,2024,4,nadia,3807,5031294.00


In [11]:
Top_User

,State,Year,Quarter,District,Pincode,RegisteredUsers
0,andaman-&-nicobar-islands,2018,1,south andaman,None,5846.00
1,andaman-&-nicobar-islands,2018,1,north and middle andaman,None,632.00
2,andaman-&-nicobar-islands,2018,1,nicobars,None,262.00
3,andaman-&-nicobar-islands,2018,1,None,744103,1608.00
4,andaman-&-nicobar-islands,2018,1,None,744101,1108.00
...,...,...,...,...,...,...
18291,west-bengal,2024,4,None,733134,169596.00
18292,west-bengal,2024,4,None,700059,169448.00
18293,west-bengal,2024,4,None,711302,146034.00
18294,west-bengal,2024,4,None,700150,142151.00


In [12]:
!pip install pymysql cryptography

import pymysql
import pandas as pd

# MySQL Connection Details
DB_NAME = "phonepay"
USER = "root"
PASSWORD = "17562dentsu"  # Replace with your MySQL password
HOST = "127.0.0.1"

# Connect to MySQL Server
conn = pymysql.connect(host=HOST, user=USER, password=PASSWORD)
cursor = conn.cursor()

# Create Database if not exists
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
cursor.execute(f"USE {DB_NAME}")

# Table Creation Queries
TABLES = {
    "Aggregated_Transactions": """
        CREATE TABLE IF NOT EXISTS Aggregated_Transactions (
            State VARCHAR(255),
            Year INT,
            Quarter INT,
            Transaction_type VARCHAR(255),
            Transaction_count BIGINT,
            Transaction_amount DOUBLE
        )
    """,
    "Aggregated_Insurance": """
        CREATE TABLE IF NOT EXISTS Aggregated_Insurance (
            State VARCHAR(255),
            Year INT,
            Quarter INT,
            Category VARCHAR(255),
            Insurance_count BIGINT,
            Insurance_amount DOUBLE
        )
    """,
    "Aggregated_Users": """
        CREATE TABLE IF NOT EXISTS Aggregated_Users (
            State VARCHAR(255),
            Year INT,
            Quarter INT,
            Registered_Users BIGINT,
            App_Opens BIGINT
        )
    """,
    "Map_Transactions": """
        CREATE TABLE IF NOT EXISTS Map_Transactions (
            State VARCHAR(255),
            Year INT,
            Quarter INT,
            Transaction_type VARCHAR(255),
            Transaction_count BIGINT,
            Transaction_amount DOUBLE
        )
    """,
    "Map_Insurance": """
        CREATE TABLE IF NOT EXISTS Map_Insurance (
            State VARCHAR(255),
            Year INT,
            Quarter INT,
            District VARCHAR(255),
            Insurance_count BIGINT,
            Insurance_amount DOUBLE
        )
    """,
    "Map_Users": """
        CREATE TABLE IF NOT EXISTS Map_Users (
            State VARCHAR(255),
            Year INT,
            Quarter INT,
            District VARCHAR(255),
            RegisteredUsers BIGINT,
            AppOpens BIGINT
        )
    """,
    "Top_Transactions": """
        CREATE TABLE IF NOT EXISTS Top_Transactions (
            State VARCHAR(255),
            Year INT,
            Quarter INT,
            District VARCHAR(255),
            Transaction_count BIGINT,
            Transaction_amount DOUBLE
        )
    """,
    "Top_Insurance": """
        CREATE TABLE IF NOT EXISTS Top_Insurance (
            State VARCHAR(255),
            Year INT,
            Quarter INT,
            District VARCHAR(255),
            Insurance_count BIGINT,
            Insurance_amount DOUBLE
        )
    """,
    "Top_Users": """
        CREATE TABLE IF NOT EXISTS Top_Users (
            State VARCHAR(255),
            Year INT,
            Quarter INT,
            District VARCHAR(255),
            Pincode VARCHAR(20),
            RegisteredUsers BIGINT
        )
    """
}

# Create Tables in Database
for table_name, table_query in TABLES.items():
    cursor.execute(table_query)

print("✅ Database and tables created successfully!")

# Function to Insert Data
def insert_dataframe(df, table_name):
    """Insert DataFrame into MySQL table."""
    placeholders = ", ".join(["%s"] * len(df.columns))
    columns = ", ".join(df.columns)
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    values = [tuple(row) for row in df.itertuples(index=False, name=None)]
    
    cursor.executemany(sql, values)
    conn.commit()
    print(f"✅ Inserted {len(df)} rows into {table_name}")

# Insert DataFrames into MySQL
insert_dataframe(Agg_Trans, "Aggregated_Transactions")
insert_dataframe(Agg_Ins, "Aggregated_Insurance")
insert_dataframe(Agg_User, "Aggregated_Users")
insert_dataframe(Map_Trans, "Map_Transactions")
insert_dataframe(Map_Insurance, "Map_Insurance")
insert_dataframe(Map_User, "Map_Users")
insert_dataframe(Top_Trans, "Top_Transactions")
insert_dataframe(Top_Ins, "Top_Insurance")
insert_dataframe(Top_User, "Top_Users")

# Close Connection
cursor.close()
print("🚀 All data inserted successfully, connection closed.")



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Database and tables created successfully!
✅ Inserted 5034 rows into Aggregated_Transactions
✅ Inserted 682 rows into Aggregated_Insurance
✅ Inserted 1008 rows into Aggregated_Users
✅ Inserted 20604 rows into Map_Transactions
✅ Inserted 13876 rows into Map_Insurance
✅ Inserted 20608 rows into Map_Users
✅ Inserted 8296 rows into Top_Transactions
✅ Inserted 5608 rows into Top_Insurance
✅ Inserted 18296 rows into Top_Users
🚀 All data inserted successfully, connection closed.
